In [219]:
import rapidfuzz
import re
import pandas as pd

from tqdm import tqdm
from rdflib import Graph, Namespace
from rdflib.namespace import XSD, RDFS, RDF, OWL

In [278]:
replacements = {
    "Peraturan Daerah Provinsi": "Perprov",
    "Peraturan Wali Kota": "Perwali",
    "Peraturan Bupati": "Perbup",
    "Peraturan Kabupaten": "Perkab",
    "Peraturan Kota": "Perkot",
    "Peraturan Menteri Koordinator Bidang": "Permenko",
    "Keputusan Menteri": "Kepmen",
    "Peraturan Menteri": "Permen",
    "Peraturan Bersama": "PB",
    "Peraturan Presiden": "Perpres",
    "Peraturan Daerah": "Perda",
    "Peraturan Pemerintah": "PP",
    "Undang-Undang Dasar": "UUD",
    "Undang-Undang": "UU",
    "Ketetapan Presiden": "Tapres",
    "Keputusan Presiden": "Keppres",
    # permenko
    "Politik, Hukum, Dan Keamanan": "Polhukam",
    "Perekonomian": "Perekonomian",
    "Kesejahteraan Rakyat": "Kesra",
    "Kemaritiman": "Maritim",
    "Pembangunan Manusia Dan Kebudayaan": "PMK",
    # kepmen, permen, pb
    "Agama": "Agama",
    "Kelautan Dan Perikanan": "KP",
    "Kehutanan": "Hut",
    "Lingkungan Hidup Dan Kehutanan": "LHK",
    "Badan Usaha Milik Negara": "BUMN",
    "Perencanaan Pembangunan Nasional/ Kepala Badan Perencanaan Pembangunan Nasional": "Bappenas",
    "Perhubungan": "Hub",
    "Pendayagunaan Aparatur Negara Dan Reformasi Birokrasi": "PANRB",
    "Pertanian": "Tan",
    "Energi Dan Sumber Daya Mineral": "ESDM",
    "Riset, Teknologi, Dan Pendidikan Tinggi": "Ristekdikti",
    "Riset Dan Teknologi": "Ristek",
    "Pemuda Dan Olahraga": "Pora",
    "Keuangan": "Keu",
    "Pariwisata": "Par",
    "Pertahanan": "Han",
    "Kebudayaan Dan Pariwisata": "Budpar",
    "Desa, Pembangunan Daerah Tertinggal, Dan Transmigrasi": "Desa",
    "Perdagangan": "Dag",
    "Riset Teknologi Dan Pendidikan Tinggi": "Ristekdikti",
    "Agraria Dan Tata Ruang/ Kepala Badan Pertanahan Nasional": "ATRBPN",
    "Pekerjaan Umum Dan Perumahan Rakyat": "PUPR",
    "Pekerjaan Umum": "PU",
    "Pendidikan Dan Kebudayaan": "Dikbud",
    "Kesehatan": "Kes",
    "Perindustrian": "Perin",
    "Tenaga Kerja Dan Transmigrasi": "Nakertrans",
    "Pariwisata Dan Ekonomi Kreatif": "Parekraf",
    "Pembangunan Daerah Tertinggal": "PDT",
    "Hukum Dan Hak Asasi Manusia": "Humham",
    "Perumahan Rakyat": "PR",
    "Pendidikan Nasional": "Dik",
    "Dalam Negeri": "Dagri",
    "Komunikasi Dan Informatika": "Kominfo",
    "Sosial": "Sos",
    "Koperasi Dan Usaha Kecil Dan Menengah": "Kopukm",
    "Pemberdayaan Perempuan Dan Perlindungan Anak": "PPPA",
    "Lingkungan Hidup": "LH",
    "Luar Negeri": "LU",
    "Sekretaris Negara": "Sesneg",
    "Ketenagakerjaan": "Naker",
    "Badan Pengawas Perdagangan Berjangka Komoditi": "Bappebti",
    "Badan Pusat Statistik": "BPS",
    "Lembaga Kebijakan Pengadaan Barang/jasa Pemerintah": "LKPP",
    "Perpustakaan Nasional": "Perpusnas",
    "Badan Tenaga Nuklir Nasional": "Batan",
    "Komisi Nasional Hak Asasi Manusia": "Komnas HAM",
    "Badan Narkotika Nasional": "BNN",
    "Badan Pengawas Tenaga Nuklir": "Bapeten",
    "Komisi Pengawas Persaingan Usaha": "KPPU",
    "Komisi Informasi": "KI",
    "Badan Nasional Penanggulangan Bencana": "BNPB",
    "Pusat Pelaporan Dan Analisis Transaksi Keuangan": "PPATK",
    "Lembaga Ilmu Pengetahuan Indonesia": "LIPI",
    "Komisi Pemilihan Umum": "KPU",
    "Badan Pengatur Hilir Minyak Dan Gas Bumi": "BPH Migas",
    "Badan Pengelola Keuangan Haji": "BPKH",
    "Badan Standardisasi Nasional": "BSN",
    "Badan Meteorologi, Klimatologi, Dan Geofisika": "BMKG",
    "Badan Nasional Penanggulangan Terorisme": "BNPT",
    "Arsip Nasional": "ANRI",
    "Lembaga Administrasi Negara": "LAN",
    "Sekretaris Jenderal Ombudsman": "Ombudsman",
    "Lembaga Perlindungan Saksi Dan Korban": "LPSK",
    "Badan Wakaf Indonesia": "BWI",
    "Dewan Perwakilan Daerah": "DPD",
    "Lembaga Penjamin Simpanan": "LPS",
    "Badan Pengkajian Dan Penerapan Teknologi": "BPPT",
    "Badan Siber Dan Sandi Negara": "BSSN",
    "Jaksa Agung": "Jagung",
    "Badan Kependudukan Dan Keluarga Berencana Nasional": "BKKBN",
    "Dewan Perwakilan Rakyat": "DPR",
    "Badan Nasional Penempatan Dan Perlindungan Tenaga Kerja Indonesia": "BNP2TKI",
    "Badan Pengawas Obat Dan Makanan": "BPOM",
    "Badan Pengawas Pemilihan Umum": "Bawaslu",
    "Komisi Aparatur Sipil Negara": "KASN",
    "Badan Nasional Pencarian Dan Pertolongan": "Basarnas",
    "Badan Kepegawaian Negara": "BKN",
    "Dewan Jaminan Sosial Nasional": "DJSN",
    "Badan Nasional Pengelola Perbatasan": "BNPP",
    "Badan Keamanan Laut": "Bakamla",
    "Badan Ekonomi Kreatif": "Bekraf",
    "Badan Pengawasan Keuangan Dan Pembangunan": "BPKP",
    "Kepala Kepolisian Negara": "Kapolri",
    "Kepolisian Negara": "Polri",
    "Komisi Pemberantasan Korupsi": "KPK",
    "Mahkamah Agung": "MA",
    "Badan Informasi Geospasial": "BIG",
    "Konsil Kedokteran Indonesia": "KKI",
    "Dewan Kawasan Perdagangan Bebas Dan Pelabuhan Bebas Batam": "PBBATAM",
    "Lembaga Penerbangan Dan Antariksa Nasional": "Lapan",
    "Sekretaris Kabinet": "Setkab",
    "Badan Koordinasi Penanaman Modal": "BKPM",
    "Otoritas Jasa Keuangan": "OJK",
    "Badan Amil Zakat Nasional": "Baznas",
    "Badan Pembinaan Ideologi Pancasila": "BPIP",
    "Badan Penyelenggara Jaminan Sosial Kesehatan Reppublik Indonesia": "BPJSKES",
    "Kepala Badan Pertanahan Nasional": "BPN",
    "Bank Indonesia": "BI",
    "Badan Pemeriksa Keuangan": "BPK",
    "Komisi Yudisial": "KY",
    "Dewan Kehormatan Penyelenggara Pemilihan Umum": "DKPP",
    "Badan Penyelenggara Jaminan Sosial Ketenagakerjaan": "BPJSTK",
    "Menteri": "Men",
    "Ketua": "",
    "Kepala": "",
    "Kota": "",
    "Kabupaten": "",
    "Dan": "",
    # provinsi
    "Jawa Timur": "Jatim",
    "Jawa Tengah": "Jateng",
    "Irian Jaya Barat": "Irba",
    "Jawa Barat": "Jabar",
    "Nusa Tenggara Timur": "NTT",
    "Sulawesi Utara": "Sulut",
    "Daerah Istimewa Yogyakarta": "Yogya",
    "Kalimantan Barat": "Kalbar",
    "Sulawesi Barat": "Sulbar",
    "Daerah Khusus Ibukota Jakarta": "DKI Jakarta",
    "Kalimantan Tengah": "Kalteng",
    "Kepulauan Riau": "Kep_Riau",
    "Kalimantan Timur": "Kaltim",
    "Sumatera Selatan": "Sumsel",
    "Sumatera Barat": "Sumbar",
    "Sulawesi Tengah": "Sulteng",
    "Kalimantan Selatan": "Kalsel",
    "Nusa Tenggara Barat": "NTB",
    "Sumatera Utara": "Sumut",
    "Sulawesi Selatan": "Sulsel",
    "Kepulauan Bangka Belitung": "Kep Babel",
    "Negara": "",
}

In [ ]:
class Rel:
    def __init__(self, name: str):
        self.name = name

class LegalDocument:
    def __init__(self, name: str):
        self.name = name
        
class LegalDocumentRef:
    def __init__(self, name: str, label: str):
        self.name = name
        self.label = label

class Person:
    def __init__(self, name: str):
        self.name = name

class Date:
    def __init__(self, value: str):
        self.value = value
        
class String:
    def __init__(self, value: str):
        self.value = value
        
class Int:
    def __init__(self, value: str):
        self.value = value

class City:
    def __init__(self, name: str):
        self.name = name

class Position:
    def __init__(self, name: str):
        self.name = name

class Place:
    def __init__(self, name: str):
        self.name = name
        
class Chapter:
    def __init__(self, name: str):
        self.name = name

class Part:
    def __init__(self, name: str):
        self.name = name
        
class Paragraph:
    def __init__(self, name: str):
        self.name = name

class Article:
    def __init__(self, name: str):
        self.name = name

class Extract:
    def __init__(self, triples):
        self.triples = triples

class Triple:
    def __init__(self, head, rel, tail):
        self.head = head
        self.rel = rel
        self.tail = tail

class Extract:
    def __init__(self, triples):
        self.triples = triples

In [276]:
# Men Hub --> Menhub; Men KP --> MENKP
def convert_string(input_string):
    pattern = r'(Men)\s+(\w+)'
    replacement = lambda match: match.group(1).upper() + match.group(2) if match.group(2).isupper() else match.group(1) + match.group(2).lower()
    output_string = re.sub(pattern, replacement, input_string).strip().replace(' ', '_')
    return output_string

# Mapping ke replacements
def replace_similar(name, replacements, threshold=10):
    for pattern, replacement in replacements.items():
        name = name.replace('Republik Indonesia', '')
        match = rapidfuzz.process.extractOne(name, [pattern], score_cutoff=threshold)
        if match:
            name = name.replace(match[0], replacement)
    
    def normalize(text):
        return re.sub(r'[-\s]+', ' ', text).strip()
    
    normalized_name = normalize(name)
    
    highest_ratio = 0
    best_match = None
    best_replacement = None
    
    for pattern, replacement in replacements.items():
        normalized_pattern = normalize(pattern)
        ratio = rapidfuzz.fuzz.QRatio(normalized_name, normalized_pattern)
        if ratio > highest_ratio and ratio >= threshold:
            highest_ratio = ratio
            best_match = normalized_pattern
            best_replacement = replacement
    print(best_match)
    print(best_replacement)
    print(name)
    
    if best_match:
        name = re.sub(r'\b' + re.escape(best_match) + r'\b', best_replacement, name, flags=re.IGNORECASE)
    
    return re.sub(r'\s+', ' ', name.replace(',', ''))

# Extract label peraturan sampai sebelum tentang --> [Nama Peraturan] Nomor [Nomor] Tahun [Tahun]
def extract_regulation(input_text):
    pattern_1 = r'(.*)(Nomor\s+\d+)\s+(?:.*?)(Tahun\s+\d{4})'  # Nomor 29 Tahun 2000
    pattern_2 = r'(.*)(Nomor\s+\d+)(?:\/[^\/]*?)*(\/\d{4})'  # Nomor 29/PP/2000
    pattern_3 = r'(.*)(Nomor\s+\d+)(?:\/[^\/]*?)*(Tahun\s+\d+)'  # Nomor 29/PP Tahun 2000
    pattern_4 = r'(Undang-Undang Dasar)(.*?)(Tahun\s+\d+)'
    
    match_1 = re.search(pattern_1, input_text)
    match_2 = re.search(pattern_2, input_text)
    match_3 = re.search(pattern_3, input_text)
    match_4 = re.search(pattern_4, input_text)
    
    if match_1 or match_3:
        match = match_1 if match_1 is not None else match_3
        regulation_type = match.group(1)
        number = match.group(2)
        year = match.group(3)
        res = f"{regulation_type} {number} {year}"
        return re.sub(r'\s+', ' ', res)
    elif match_2:
        regulation_type = match_2.group(1)
        number = match_2.group(2)
        year = "Tahun " + match_2.group(3)[1:]
        res = f"{regulation_type} {number} {year}"
        return re.sub(r'\s+', ' ', res)
    elif match_4:
        regulation_type = match_4.group(1)
        conj = match_4.group(2)
        year = match_4.group(3)
        res = f"{regulation_type} {conj} {year}"
        return re.sub(r'\s+', ' ', res)
    else:
        return input_text

# [Nama Peraturan, Nomor XX Tahun YYYY]
def split_regulation(text):
    match = re.search(r'(.*?)(Nomor .*|Tahun .*)', text, re.IGNORECASE)
    if match:
        part1 = match.group(1).strip()
        part2 = match.group(2).strip()
        return [part1, part2]
    else:
        return [text, '']

# Nomor XX Tahun YYYY --> YYYY_XX
def convert_nomor_tahun(text):
    match = re.search(r'Nomor (\d+)\s+Tahun (\d+)', text, re.IGNORECASE)
    match_2 = re.search(r'Tahun (\d+)', text, re.IGNORECASE)
    if match:
        number = match.group(1)
        year = match.group(2)
        return f"{year}_{number}"
    elif match_2:
        year = match_2.group(1)
        return year
    else:
        return text

# Nama Peraturan --> id
def convert_nama_peraturan(text):
    converted_text = convert_string(replace_similar(text, replacements))
    return converted_text


def reg_to_id(text):
    label = extract_regulation(text)
    regulatory, num = split_regulation(label)
    a = convert_nama_peraturan(regulatory)
    b = convert_nomor_tahun(num)
    reg_id = f'{a}_{b}' if b != '' else a
    return label, reg_id

In [ ]:
def convert_label(label):
    # BAB III
    match = re.match(r'^BAB\s+(\w+)$', label)
    if match:
        return match.group(1)
    
    # Article 18
    match = re.match(r'^Article\s+(\d+)$', label)
    if match:
        return match.group(1)
    
    # Bagian Kesatu BAB III
    match = re.match(r'^Bagian\s+(\w+) BAB (\w+)$', label)
    if match:
        return f"{match.group(2)}_{match.group(1).upper()}"
    
    # Paragraf 3 Bagian Kedua BAB III
    match = re.match(r'^Paragraf\s+(\d+)\s+Bagian\s+(\w+)\s+BAB\s+(\w+)$', label)
    if match:
        return f"{match.group(3)}_{match.group(2).upper()}_{match.group(1)}"
    
    return label

def parse_input_string(input_str):
    input_str = input_str.replace('extract = ', '')
    input_str = input_str.replace(',])', '')
    
    triples = []
    pattern = re.compile(
        r"Triple\((?P<head_type>\w+)\('(?P<head_value>.*?)'\), "
        r"Rel\('(?P<rel>.*?)'\), "
        r"(?P<tail_type>\w+)\('(?P<tail_value>.*?)'\)\)"
    )
    
    for match in pattern.finditer(input_str):
        head_type = match.group('head_type')
        head_value = match.group('head_value')
        rel = match.group('rel')
        tail_type = match.group('tail_type')
        tail_value = match.group('tail_value')
        
        if head_type in ('LegalDocument'):
            label, name = reg_to_id(head_value)
            head = LegalDocument(name)
        else:
            continue
        
        if tail_type == 'Place':
            tail = Place(tail_value)
        elif tail_type == 'Date':
            tail = Date(tail_value)
        elif tail_type == 'String':
            tail = String(tail_value)
        elif tail_type == 'Int':
            tail = Int(tail_value)
        elif tail_type == 'City':
            tail = City(tail_value)
        elif tail_type == 'Person':
            tail = Person(tail_value)
        elif tail_type == 'Position':
            tail = Position(tail_value)
        elif tail_type == 'LegalDocument':
            if ' ' not in tail_value:
                tail = LegalDocument(tail_value)
            else:
                label, name = reg_to_id(tail_value)
                tail = LegalDocumentRef(name, label)
        else:
            continue
        
        triples.append(Triple(head, Rel(rel), tail))
    
    return Extract(triples)

def parse_input_string_2(input_str):
    input_str = input_str.replace("\n", "").replace("    ", "").replace("extract = Extract([", "extract = Extract([").replace("]))", ")),])")
    input_str = input_str.replace('extract = ', '')
    input_str = input_str.replace(',])', '')
    
    triples = []
    pattern = re.compile(
        r"Triple\((?P<head_type>\w+)\('(?P<head_value>.*?)'\),"
        r"Rel\('(?P<rel>.*?)'\),"
        r"(?P<tail_type>\w+)\('(?P<tail_value>.*?)'\)\)"
    )
    
    for match in pattern.finditer(input_str):
        head_type = match.group('head_type')
        head_value = match.group('head_value')
        rel = match.group('rel')
        tail_type = match.group('tail_type')
        tail_value = match.group('tail_value')
        
        if head_type in ('LegalDocument'):
            label, name = reg_to_id(head_value)
            head = LegalDocument(name)
        else:
            continue
        
        if tail_type == 'Place':
            tail = Place(tail_value)
        elif tail_type == 'Date':
            tail = Date(tail_value)
        elif tail_type == 'String':
            tail = String(tail_value)
        elif tail_type == 'Int':
            tail = Int(tail_value)
        elif tail_type == 'City':
            tail = City(tail_value)
        elif tail_type == 'Person':
            tail = Person(tail_value)
        elif tail_type == 'Position':
            tail = Position(tail_value)
        elif tail_type == 'LegalDocument':
            if ' ' not in tail_value:
                tail = LegalDocument(tail_value)
            else:
                label, name = reg_to_id(tail_value)
                tail = LegalDocumentRef(name, label)
        else:
            continue
        
        triples.append(Triple(head, Rel(rel), tail))
    
    return Extract(triples)

def parse_input_string_3(input_str):
    input_str = input_str.replace('extract = ', '')
    input_str = input_str.replace(',])', '')
    
    triples = []
    pattern = re.compile(
        r"Triple\((?P<head_type>\w+)\('(?P<head_value>.*?)'\), "
        r"Rel\('(?P<rel>.*?)'\), "
        r"(?P<tail_type>\w+)\('(?P<tail_value>.*?)'\)\)"
    )
    
    for match in pattern.finditer(input_str):
        head_type = match.group('head_type')
        head_value = match.group('head_value')
        rel = match.group('rel')
        tail_type = match.group('tail_type')
        tail_value = match.group('tail_value')
        
        if head_type in 'LegalDocument':
            head = LegalDocument(head_value)
        elif head_type == 'Chapter':
            head = Chapter(head_value)
        elif head_type == 'Part':
            head = Part(head_value)
        elif head_type == 'Paragraph':
            head = Paragraph(head_value)
        elif head_type == 'Article':
            head = Article(head_value)
        else:
            continue
        
        if tail_type == 'Place':
            tail = Place(tail_value)
        elif tail_type == 'Date':
            tail = Date(tail_value)
        elif tail_type == 'String':
            tail = String(tail_value)
        elif tail_type == 'Int':
            tail = Int(tail_value)
        elif tail_type == 'City':
            tail = City(tail_value)
        elif tail_type == 'Person':
            tail = Person(tail_value)
        elif tail_type == 'Position':
            tail = Position(tail_value)
        elif tail_type == 'Chapter':
            tail = Chapter(tail_value)
        elif tail_type == 'Part':
            tail = Part(tail_value)
        elif tail_type == 'Paragraph':
            tail = Paragraph(tail_value)
        elif tail_type == 'Article':
            tail = Article(tail_value)
        elif tail_type == 'LegalDocument':
            if ' ' not in tail_value:
                tail = LegalDocument(tail_value)
            else:
                label, name = reg_to_id(tail_value)
                tail = LegalDocumentRef(name, label)
        else:
            continue
        
        triples.append(Triple(head, Rel(rel), tail))
    
    return Extract(triples)


def format_name(name):
    name = re.sub(r'\s+', '_', name)
    name = name.replace(',', '').replace('.', '').replace('(', '').replace(')', '').title()
    return name


def format_name_body(label):
    match = re.match(r'^(BAB|Bagian|Paragraf|Pasal)\s+(\w+)$', label)
    if match:
        return match.group(2)
    match = re.match(r'^(BAB|Bagian|Paragraf|Pasal)\s+(\d+)$', label)
    if match:
        return match.group(2)
    match = re.match(r'^Bagian\s+(\w+)\s+BAB\s+(\w+)$', label)
    if match:
        return f"{match.group(2)}_{match.group(1).upper()}"
    match = re.match(r'^Paragraf\s+(\d+)\s+Bagian\s+(\w+)\s+BAB\s+(\w+)$', label)
    if match:
        return f"{match.group(3)}_{match.group(2).upper()}_{match.group(1)}"
    return format_name(label)


def get_prefixed_name(reg, obj):
    if isinstance(obj, City):
        return f"City_{format_name(obj.name)}"
    elif isinstance(obj, Person):
        return f"Person_{format_name(obj.name)}"
    elif isinstance(obj, Position):
        return f"Position_{format_name(obj.name)}"
    elif isinstance(obj, Chapter):
        return f"{reg}_Chapter_{format_name_body(obj.name)}"
    elif isinstance(obj, Part):
        return f"{reg}_Part_{format_name_body(obj.name)}"
    elif isinstance(obj, Paragraph):
        return f"{reg}_Paragraph_{format_name_body(obj.name)}"
    elif isinstance(obj, Article):
        return f"{reg}_Article_{format_name_body(obj.name)}"
    elif isinstance(obj, (LegalDocumentRef, LegalDocument)):
        return f"{(obj.name)}"
    return format_name(obj.name)


def get_relation_name(name):
    name = name.replace('hasName', 'name')
    return name

## RDF to TTL

In [223]:
prefix = '''@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix wd: <https://www.wikidata.org/wiki/> .
@prefix lexid-s: <https://w3id.org/lex-id/schema/> .
@prefix lexid: <https://w3id.org/lex-id/data/> .

'''

In [224]:
def is_valid_triples(triple_str):
    ttl = prefix
    ttl += triple_str
    
    g = Graph()
    DBO = Namespace("http://dbpedia.org/ontology/")
    DCT = Namespace("http://purl.org/dc/terms/")
    WD = Namespace("https://www.wikidata.org/wiki/")
    LEXID_S = Namespace("https://w3id.org/lex-id/schema/")
    LEXID = Namespace("https://w3id.org/lex-id/data/")

    g.bind("xsd", XSD)
    g.bind("rdfs", RDFS)
    g.bind("rdf", RDF)
    g.bind("dbo", DBO)
    g.bind("dct", DCT)
    g.bind("owl", OWL)
    g.bind("wd", WD)
    g.bind("lexid-s", LEXID_S)
    g.bind("lexid", LEXID)
    
    try:
        g.parse(data=ttl, format="turtle")
        return True
    except:
        return False
    

def is_valid_ttl(triple_str):
    g = Graph()
    DBO = Namespace("http://dbpedia.org/ontology/")
    DCT = Namespace("http://purl.org/dc/terms/")
    WD = Namespace("https://www.wikidata.org/wiki/")
    LEXID_S = Namespace("https://w3id.org/lex-id/schema/")
    LEXID = Namespace("https://w3id.org/lex-id/data/")

    g.bind("xsd", XSD)
    g.bind("rdfs", RDFS)
    g.bind("rdf", RDF)
    g.bind("dbo", DBO)
    g.bind("dct", DCT)
    g.bind("owl", OWL)
    g.bind("wd", WD)
    g.bind("lexid-s", LEXID_S)
    g.bind("lexid", LEXID)
    
    try:
        g.parse(data=triple_str, format="turtle")
        return True
    except:
        return False

In [225]:
error = pd.DataFrame()
error = pd.DataFrame(columns=['regulatory', 'name', 'ttl'])

In [226]:
def check_rdf(reg, name, cell):
    ttl = prefix
    
    if not pd.isna(cell):
        triples = cell.split('\n')
        
        for j in range(len(triples)):
            triple = triples[j]
            
            if is_valid_triples(triple):
                ttl += triple + '\n'
                
            if not is_valid_triples(triple) and triple.count(':') >= 3 and 'Name of' not in triple:
                
                #case triple kepisah line
                if j + 1 < len(triples) and is_valid_triples(triples[j] + triples[j+1]):
                    triple = triples[j] + triples[j+1]
                
                #case ada double " contoh: lexid:Perkot_Denpasar_2006_1 lexid-s:hasEnactionDate ""2006-01-01""^^xsd:date .
                if '""' in triple:
                    triple = triple.replace('""', '"')
                
                #case enitity ada '
                if "'" in triple:
                    parts = re.split(r'(".*?")', triple)
                    result = [part.replace("'", "") if not part.startswith('"') else part for part in parts]
                    triple = ''.join(result)
                
                #case enitity ada /
                if "/" in triple:
                    parts = re.split(r'(".*?")', triple)
                    result = [part.replace("/", "_") if not part.startswith('"') else part for part in parts]
                    triple = ''.join(result)
                
                #case enitity ada /
                if "_&_" in triple:
                    parts = re.split(r'(".*?")', triple)
                    result = [part.replace("_&_", "_dan_") if not part.startswith('"') else part for part in parts]
                    triple = ''.join(result)
                
                # cek pattern ada spasi di antara entity setelah titik contoh: lexid:Person_R. Nuriyana
                pattern1 = r'(\. )(?=(?:[^"]*"[^"]*")*[^"]*$)'
                triple = re.sub(pattern1, '_', triple)
                
                #case: lexid:Position_Menteri_Kehakiman_a.i. rdfs:label "Menteri Kehakiman a.i."^^xsd:string .
                if '_rdf' in triple:
                    triple = triple.replace('_rdf', ' rdf')
                if "." in triple[:-1]:
                    parts = re.split(r'(".*?")', triple[:-1])
                    result = [part.replace(".", "_") if not part.startswith('"') else part for part in parts]
                    triple = ''.join(result) + '.'
                
                #case: lexid:Person_Sofyan_A_Dj LIABLE
                index = triple.find(' rdf')
                if index != -1:
                    index_lexid = triple.find('lexid', 0, index)
                    if index_lexid != -1:
                        triple = triple[:index].replace(' ', '_') + triple[index:]

                index_lexid_s = triple.find('lexid-s')
                if index_lexid_s != -1:
                    index_lexid = triple.find('lexid', index_lexid_s + len('lexid-s'))
                    if index_lexid != -1:
                        triple = triple[:index_lexid] + triple[index_lexid:].replace(' ', '_')
                        
                if 'Lexid' in triple:
                    triple = triple.replace('Lexid', 'lexid')
                    
                # kurang titik
                if triple[-1] != '.':
                    triple += '.'
                

                if is_valid_triples(triple):
                    ttl += triple + '\n'
                   
    try:
        if is_valid_ttl(ttl):
            return ttl
    except:
        l = len(error)
        error.at[l, 'regulatory'] = reg
        error.at[l, 'name'] = name
        error.at[l, 'ttl'] = cell
        return prefix

## Closing

In [11]:
cc_gemma = pd.read_csv('results/new/CC_gemma.csv')
cc_llama = pd.read_csv('results/new/CC_llama.csv')
cc_phi = pd.read_csv('results/new/CC_phi.csv')

In [ ]:
def txt_to_rdf(inp):
    extract = parse_input_string(inp)
    rdf_triples = []
    res = ''

    for triple in extract.triples:
        head_lexid = f"lexid:{get_prefixed_name(triple.head)}"
        rel_lexid = f"lexid-s:{get_relation_name(triple.rel.name)}"
        
        if rel_lexid == 'lexid-s:label':
            rel_lexid = 'rdfs:label'

        if isinstance(triple.tail, (LegalDocument, LegalDocumentRef, Place, City, Person, Position)):
            tail_lexid = f"lexid:{get_prefixed_name(triple.tail)}"
            rdf_triples.append(f"{head_lexid} {rel_lexid} {tail_lexid} .")

            if isinstance(triple.tail, City):
                rdf_triples.append(f"{tail_lexid} rdf:type lexid-s:City .")
            elif isinstance(triple.tail, Person):
                rdf_triples.append(f"{tail_lexid} rdf:type lexid-s:Person .")
            elif isinstance(triple.tail, Position):
                rdf_triples.append(f"{tail_lexid} rdf:type lexid-s:Position .")
            elif isinstance(triple.tail, LegalDocumentRef):
                rdf_triples.append(f"{tail_lexid} rdf:type lexid-s:LegalDocument .")

            if isinstance(triple.tail, (City, Person, Position)):
                rdf_triples.append(f"{tail_lexid} rdfs:label \"{triple.tail.name}\"^^xsd:string .")
            elif isinstance(triple.tail, LegalDocumentRef):
                rdf_triples.append(f"{tail_lexid} lexid-s:LegalBasisOf {head_lexid} .")
                rdf_triples.append(f"{tail_lexid} rdfs:label \"{triple.tail.label}\"^^xsd:string .")

        elif isinstance(triple.tail, Date):
            rdf_triples.append(f"{head_lexid} {rel_lexid} \"{triple.tail.value}\"^^xsd:date .")
        elif isinstance(triple.tail, String):
            rdf_triples.append(f"{head_lexid} {rel_lexid} \"{triple.tail.value}\"^^xsd:string .")
        elif isinstance(triple.tail, Int):
            rdf_triples.append(f"{head_lexid} {rel_lexid} \"{triple.tail.value}\"^^xsd:int .")
            
        if rel_lexid == 'lexid-s:regulationYear':
            rdf_triples.append(f"{head_lexid} rdf:type lexid-s:LegalDocument .")

    for triple in rdf_triples:
        res += triple + '\n'
    
    return res

def rdf_to_ttl_1(name, filename):
    df = pd.read_csv(filename)
    result = pd.DataFrame()
    result = pd.DataFrame(columns=['regulatory', '1', '2', 'ttl_1', 'ttl_2'])
    
    target_filename = filename.replace('.csv', '_post.csv')
    
    cnt = 0
    
    for i in tqdm(range(len(df))):
        reg = df.at[i, 'regulatory']
        out_1 = df.at[i, '1']
        out_2 = df.at[i, '2']
        
        if pd.isna(out_1):
            res_1 = ''
        else:
            res_1 = txt_to_rdf(out_1)
            
        if pd.isna(out_2):
            res_2 = ''
        else:
            res_2 = txt_to_rdf(out_2)
        
        ttl_1 = check_rdf(reg, f'{name} 1', res_1)
        ttl_2 = check_rdf(reg, f'{name} 2', res_2)
            
        result.at[i, 'regulatory'] = reg
        result.at[i, '1'] = res_1
        result.at[i, '2'] = res_2
        result.at[i, 'ttl_1'] = ttl_1
        result.at[i, 'ttl_2'] = ttl_2
        
        print(cnt, reg)
        cnt += 1

        if cnt % 20 == 0:
            result.to_csv(target_filename)
            
    result.to_csv(target_filename)

In [ ]:
rdf_to_ttl_1('cc_gemma', 'results/new/CC_gemma.csv')

In [ ]:
rdf_to_ttl_1('cc_llama', 'results/new/CC_llama.csv')

In [ ]:
rdf_to_ttl_1('cc_phi', 'results/new/CC_phi.csv')

## Opening

In [246]:
oc_gemma = pd.read_csv('results/new/OC_gemma.csv')
oc_llama = pd.read_csv('results/new/OC_llama.csv')
oc_phi = pd.read_csv('results/new/OC_phi.csv')

In [247]:
oc_label_gemma = pd.read_csv('results/new/OC_label_gemma.csv')
oc_label_llama = pd.read_csv('results/new/OC_label_llama.csv')
oc_label_phi = pd.read_csv('results/new/OC_label_phi.csv')

In [ ]:
def txt_to_rdf_2(reg, inp):
    extract = parse_input_string_2(inp)
    rdf_triples = []
    res = ''
    
    for triple in extract.triples:
        h = LegalDocument(reg)
        head_lexid = f"lexid:{get_prefixed_name(h)}"
        rel_lexid = f"rdfs:{get_relation_name(triple.rel.name)}"

        if isinstance(triple.tail, (LegalDocument, LegalDocumentRef, Place, City, Person, Position)):
            tail_lexid = f"lexid:{get_prefixed_name(triple.tail)}"
            rdf_triples.append(f"{head_lexid} {rel_lexid} {tail_lexid} .")

            if isinstance(triple.tail, City):
                rdf_triples.append(f"{tail_lexid} rdf:type lexid-s:City .")
            elif isinstance(triple.tail, Person):
                rdf_triples.append(f"{tail_lexid} rdf:type lexid-s:Person .")
            elif isinstance(triple.tail, Position):
                rdf_triples.append(f"{tail_lexid} rdf:type lexid-s:Position .")
            elif isinstance(triple.tail, LegalDocumentRef):
                rdf_triples.append(f"{tail_lexid} rdf:type lexid-s:LegalDocument .")

            if isinstance(triple.tail, (City, Person, Position)):
                rdf_triples.append(f"{tail_lexid} rdfs:label \"{triple.tail.name}\"^^xsd:string .")
            elif isinstance(triple.tail, LegalDocumentRef):
                rdf_triples.append(f"{tail_lexid} lexid-s:LegalBasisOf {head_lexid} .")
                rdf_triples.append(f"{tail_lexid} rdfs:label \"{triple.tail.label}\"^^xsd:string .")

        elif isinstance(triple.tail, Date):
            rdf_triples.append(f"{head_lexid} {rel_lexid} \"{triple.tail.value}\"^^xsd:date .")
        elif isinstance(triple.tail, String):
            rdf_triples.append(f"{head_lexid} {rel_lexid} \"{triple.tail.value}\"^^xsd:string .")
        elif isinstance(triple.tail, Int):
            rdf_triples.append(f"{head_lexid} {rel_lexid} \"{triple.tail.value}\"^^xsd:int .")
            
        if rel_lexid == 'lexid-s:regulationYear':
            rdf_triples.append(f"{head_lexid} rdf:type lexid-s:LegalDocument .")

    for triple in rdf_triples:
        res += triple + '\n'
    
    return res

def get_label_oc(reg, inp):
    label = txt_to_rdf(inp)
    if label == '':
        label = txt_to_rdf_2(reg, inp)
    return label.strip()

def rdf_to_ttl_2(name, filename, file_label):
    df = pd.read_csv(filename)
    df_label = pd.read_csv(file_label)
    result = pd.DataFrame()
    result = pd.DataFrame(columns=['regulatory', '1', '2', 'ttl_1', 'ttl_2'])
    
    target_filename = filename.replace('.csv', '_post_0.csv')
    
    cnt = 0
    
    for i in tqdm(range(len(df))):
        reg = df.at[i, 'regulatory']
        out_1_1 = df.at[i, '1_1']
        out_1_2 = df.at[i, '1_2']
        out_1_3 = df_label.at[i, '1_3']
        out_2_1 = df.at[i, '2_1']
        out_2_2 = df.at[i, '2_2']
        out_2_3 = df_label.at[i, '2_3']
        
        if pd.isna(out_1_1):
            res_1_1 = ''
        else:
            res_1_1 = txt_to_rdf(out_1_1)
            
        if pd.isna(out_1_2):
            res_1_2 = ''
        else:
            res_1_2 = txt_to_rdf(out_1_2)
            
        if pd.isna(out_1_3):
            res_1_3 = ''
        else:
            res_1_3 = get_label_oc(reg, out_1_3)
            
        if pd.isna(out_2_1):
            res_2_1 = ''
        else:
            res_2_1 = txt_to_rdf(out_2_1)
            
        if pd.isna(out_2_2):
            res_2_2 = ''
        else:
            res_2_2 = txt_to_rdf(out_2_2)
            
        if pd.isna(out_2_3):
            res_2_3 = ''
        else:
            res_2_3 = get_label_oc(reg, out_2_3)
            
        res_1 = res_1_1 + res_1_2 + res_1_3
        res_2 = res_2_1 + res_2_2 + res_2_3
        
        ttl_1 = check_rdf(reg, f'{name} 1', res_1)
        ttl_2 = check_rdf(reg, f'{name} 2', res_2)
            
        result.at[i, 'regulatory'] = reg
        result.at[i, '1'] = res_1
        result.at[i, '2'] = res_2
        result.at[i, 'ttl_1'] = ttl_1
        result.at[i, 'ttl_2'] = ttl_2
        
        print(cnt, reg)
        cnt += 1

        if cnt % 20 == 0:
            result.to_csv(target_filename)
            
    result.to_csv(target_filename)

In [ ]:
rdf_to_ttl_2('oc_gemma', 'results/new/OC_gemma.csv', 'results/new/OC_label_gemma.csv')

In [ ]:
rdf_to_ttl_2('oc_llama', 'results/new/OC_llama.csv', 'results/new/OC_label_llama.csv')

In [ ]:
rdf_to_ttl_2('oc_phi', 'results/new/OC_phi.csv', 'results/new/OC_label_phi.csv')

## Body

In [230]:
bc_gemma = pd.read_csv('results/new/BC_gemma.csv')
bc_llama = pd.read_csv('results/new/BC_llama.csv')
bc_phi = pd.read_csv('results/new/BC_phi.csv')

In [ ]:
def txt_to_rdf_3(reg, inp):
    extract = parse_input_string_3(inp)
    rdf_triples = []
    res = ''

    for triple in extract.triples:
        head_lexid = f"lexid:{get_prefixed_name(reg, triple.head)}"
        rel_lexid = f"lexid-s:{get_relation_name(triple.rel.name)}"

        if isinstance(triple.tail, (LegalDocument, LegalDocumentRef, Place, City, Person, Position, Chapter, Part, Paragraph, Article)):
            tail_lexid = f"lexid:{get_prefixed_name(reg, triple.tail)}"
            rdf_triples.append(f"{head_lexid} {rel_lexid} {tail_lexid} .")

            if isinstance(triple.tail, City):
                rdf_triples.append(f"{tail_lexid} rdf:type lexid-s:City .")
            elif isinstance(triple.tail, Person):
                rdf_triples.append(f"{tail_lexid} rdf:type lexid-s:Person .")
            elif isinstance(triple.tail, Position):
                rdf_triples.append(f"{tail_lexid} rdf:type lexid-s:Position .")
            elif isinstance(triple.tail, Article):
                rdf_triples.append(f"{tail_lexid} lexid-s:name \"{triple.tail.name}\"^^xsd:string .")
            elif isinstance(triple.tail, LegalDocumentRef):
                rdf_triples.append(f"{tail_lexid} rdf:type lexid-s:LegalDocument .")

            if isinstance(triple.tail, (City, Person, Position)):
                rdf_triples.append(f"{tail_lexid} rdfs:label \"{triple.tail.name}\"^^xsd:string .")
            elif isinstance(triple.tail, LegalDocumentRef):
                rdf_triples.append(f"{tail_lexid} lexid-s:LegalBasisOf {head_lexid} .")
                rdf_triples.append(f"{tail_lexid} rdfs:label \"{triple.tail.label}\"^^xsd:string .")

        elif isinstance(triple.tail, Date):
            rdf_triples.append(f"{head_lexid} {rel_lexid} \"{triple.tail.value}\"^^xsd:date .")
        elif isinstance(triple.tail, String):
            rdf_triples.append(f"{head_lexid} {rel_lexid} \"{triple.tail.value}\"^^xsd:string .")
        elif isinstance(triple.tail, Int):
            rdf_triples.append(f"{head_lexid} {rel_lexid} \"{triple.tail.value}\"^^xsd:int .")
            
            
        if rel_lexid == 'lexid-s:regulationYear':
            rdf_triples.append(f"{head_lexid} rdf:type lexid-s:LegalDocument .")

    for triple in rdf_triples:
        res += triple + '\n'
    
    return res

def rdf_to_ttl_3(name, filename):
    df = pd.read_csv(filename)
    result = pd.DataFrame()
    result = pd.DataFrame(columns=['regulatory', '1', '2', 'ttl_1', 'ttl_2'])
    
    target_filename = filename.replace('.csv', '_fin_0.csv')
    
    cnt = 0
    
    for i in tqdm(range(len(df))):
        reg = df.at[i, 'regulatory']
        out_1 = df.at[i, '1']
        out_2 = df.at[i, '2']
        
        if pd.isna(out_1):
            res_1 = ''
        else:
            res_1 = txt_to_rdf_3(reg, out_1)
            
        if pd.isna(out_2):
            res_2 = ''
        else:
            res_2 = txt_to_rdf_3(reg, out_2)
        
        ttl_1 = check_rdf(reg, f'{name} 1', res_1)
        ttl_2 = check_rdf(reg, f'{name} 2', res_2)
            
        result.at[i, 'regulatory'] = reg
        result.at[i, '1'] = res_1
        result.at[i, '2'] = res_2
        result.at[i, 'ttl_1'] = ttl_1
        result.at[i, 'ttl_2'] = ttl_2
        
        print(cnt, reg)
        cnt += 1

        if cnt % 20 == 0:
            result.to_csv(target_filename)
            
    result.to_csv(target_filename)

In [ ]:
rdf_to_ttl_3('bc_gemma', 'results/new/BC_gemma.csv')

In [ ]:
rdf_to_ttl_3('bc_llama', 'results/new/BC_llama.csv')

In [ ]:
rdf_to_ttl_3('bc_phi', 'results/new/BC_phi.csv')